<a href="https://colab.research.google.com/github/songqsh/MA2210/blob/main/src/simplex_wg_dual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplex method for the WG-dual

We are going to run simplex tableau method for the dual of the example - WG:
$$\min w = 4 y_1  + 12 y_2 + 18 y_3$$
s.t.
$$\begin{array}{lll}
y_1 & & + 3y_3 & \ge 3 \\
& 2y_2 &+2y_3 & \ge 5
\end{array}
$$
for $y_i\ge 0$.

In [7]:
# import package
import numpy as np
import numpy.linalg as la
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
import pandas as pd
import warnings
#ignore by message
warnings.filterwarnings("ignore", message="divide by zero encountered in true_divide")


## Matrix in numpy

In [8]:
# input augmented matrix with big M
M = 100.0

AM = np.array([
               [-1, 4, 12, 18, 0, M, 0, M, 0], 
               [0, 1, 0, 3, -1, 1, 0, 0, 3], 
               [0, 0, 2, 2, 0, 0, -1, 1, 5]
               ], dtype = float)
pd.DataFrame(AM)
#print(pd.AM) # print matrix

,0,1,2,3,4,5,6,7,8
0,-1.0,4.0,12.0,18.0,0.0,100.0,0.0,100.0,0.0
1,0.0,1.0,0.0,3.0,-1.0,1.0,0.0,0.0,3.0
2,0.0,0.0,2.0,2.0,0.0,0.0,-1.0,1.0,5.0


## Pivotizing

Pivotizing with $(i,j)$-entry means converting the matrix by multiple EROS so that the resulting matrix has all zeros in $j$-column, but one in $(i,j)$-entry. This is the key step in updating basic feasible solution.

In [9]:
#pivotize with (i,j)
def pivot(A, i, j):
  A[i] = A[i]/A[i,j] #scale to get one in (i,j)
  n_rows, _ = A.shape
  for k in range(n_rows):
    if k==i:
      continue # skip i-row
    A[k] = A[k] - A[i]*A[k,j] # replacement to get zero
    

## Simplex tableau

In [10]:
AM # print original augmented matrix

array([[-1.00, 4.00, 12.00, 18.00, 0.00, 100.00, 0.00, 100.00, 0.00],
       [0.00, 1.00, 0.00, 3.00, -1.00, 1.00, 0.00, 0.00, 3.00],
       [0.00, 0.00, 2.00, 2.00, 0.00, 0.00, -1.00, 1.00, 5.00]])

In [11]:
pivot(AM, 1, 5)
AM # print

array([[-1.00, -96.00, 12.00, -282.00, 100.00, 0.00, 0.00, 100.00,
        -300.00],
       [0.00, 1.00, 0.00, 3.00, -1.00, 1.00, 0.00, 0.00, 3.00],
       [0.00, 0.00, 2.00, 2.00, 0.00, 0.00, -1.00, 1.00, 5.00]])

In [12]:
pivot(AM, 2, 7)
AM #print

array([[-1.00, -96.00, -188.00, -482.00, 100.00, 0.00, 100.00, 0.00,
        -800.00],
       [0.00, 1.00, 0.00, 3.00, -1.00, 1.00, 0.00, 0.00, 3.00],
       [0.00, 0.00, 2.00, 2.00, 0.00, 0.00, -1.00, 1.00, 5.00]])

In [13]:
pivot(AM, 1, 3)
AM

array([[-1.00, 64.67, -188.00, 0.00, -60.67, 160.67, 100.00, 0.00,
        -318.00],
       [0.00, 0.33, 0.00, 1.00, -0.33, 0.33, 0.00, 0.00, 1.00],
       [0.00, -0.67, 2.00, 0.00, 0.67, -0.67, -1.00, 1.00, 3.00]])

In [14]:
pivot(AM, 2,2)
AM

array([[-1.00, 2.00, 0.00, 0.00, 2.00, 98.00, 6.00, 94.00, -36.00],
       [0.00, 0.33, 0.00, 1.00, -0.33, 0.33, 0.00, 0.00, 1.00],
       [0.00, -0.33, 1.00, 0.00, 0.33, -0.33, -0.50, 0.50, 1.50]])

Since the 0-row has no negative number, the last entry of the first row is the maximum value of -w, which is -36. This implies min of w is 36